In [1]:
import os
from pathlib import Path
import sys

sys.path.append(str(Path(".").absolute().parent))
sys.path.append(str(Path(".").absolute().parent) + "/utils")

import argparse
import pipeline_utils
import pandas as pd

In [2]:
PARAMS = pipeline_utils.yaml_import('settings.yml')

ENV = PARAMS['env_flag']

PROJECT_ID = PARAMS['envs'][ENV]['PROJECT_ID']
REGION = PARAMS['envs'][ENV]['REGION']
BASE_IMAGE = PARAMS['envs'][ENV]['BASE_IMAGE']
MLFLOW_IMAGE = PARAMS['envs'][ENV]['MLFLOW_IMAGE']

PIPELINE_ROOT = PARAMS['envs'][ENV]['PIPELINE_ROOT']
PIPELINE_NAME = PARAMS['envs'][ENV]['PIPELINE_NAME']
PIPELINE_JSON = PARAMS['envs'][ENV]['PIPELINE_JSON']
TMP_PIPELINE_JSON = os.path.join("/tmp", PIPELINE_JSON)


TRAINING_TABLE_NAME = PARAMS['envs'][ENV]['TRAINING_TABLE_NAME']
TRAINING_DATA_BQ_QUERY = f'select * from {TRAINING_TABLE_NAME}'

MLFLOW_EXP_NAME = PARAMS['envs'][ENV]['MLFLOW_EXP_NAME']
MODEL_REGISTRY_NAME = PARAMS['envs'][ENV]['MODEL_REGISTRY_NAME']

SERVICE_ACCOUNT = PARAMS['envs'][ENV]['SERVICE_ACCOUNT']

# TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
 
# Matches on non-word, non-regular-punctuation characters.
MATCHER = r"""[^a-zA-Z0-9'"!@#$%\^&*()\[\]{}:;<>?,.-=_+ ]+""" 

CLUB_THRESH_PATH = PARAMS['envs'][ENV]['CLUB_THRESH_PATH']
LATEST_NOSALES_MODEL_PATH = PARAMS['envs'][ENV]['LATEST_NOSALES_MODEL_PATH']
LATEST_PIPELINE_PATH = PARAMS['envs'][ENV]['LATEST_PIPELINE_PATH']
RUN_PIPELINE = PARAMS['envs'][ENV]['RUN_PIPELINE']
print(f"ENV: {ENV}, \nPROJECT_ID: {PROJECT_ID}, \nBASE_IMAGE: {BASE_IMAGE}, \nMLFLOW_IMAGE: {MLFLOW_IMAGE}, \nPIPELINE_NAME: {PIPELINE_NAME}, \nPIPELINE_JSON: {PIPELINE_JSON}")


ENV: dev, 
PROJECT_ID: wmt-mlp-p-oyi-ds-or-oyi-dsns, 
BASE_IMAGE: gcr.io/wmt-mlp-p-oyi-ds-or-oyi-dsns/oyi-vertex-pipeline-dev:latest, 
MLFLOW_IMAGE: gcr.io/wmt-mlp-p-oyi-ds-or-oyi-dsns/mlflow-image-dev:latest, 
PIPELINE_NAME: oyi-nosales-model-pipeline-dev, 
PIPELINE_JSON: oyi-nosales-model-pipeline-dev.json


In [3]:
# def check(path):
#     import os
#     import pandas as pd
#     file_path = os.path.join(path, "club_thresh_chain.csv")
#     print(file_path)
#     df_cancelled_thresh = pd.read_csv(file_path)
#     # df_cancelled_thresh.columns = ['club_nbr', 'cancelled_club_thresh', 'nosales_club_thresh']
#     print(df_cancelled_thresh)
#     # columns = ['club_nbr','nosales_club_thresh']
# check(CLUB_THRESH_PATH)

In [4]:
path = CLUB_THRESH_PATH
file_path = os.path.join(path, "club_thresh_chain.csv")
df_no_sale = pd.read_csv(file_path)
print(df_no_sale.count())

club_nbr                 599
cancelled_club_thresh    599
nosales_club_thresh      599
dtype: int64


In [ ]:
df_no_sale.info()
df_no_sale

In [ ]:
df_no_sale[(df_no_sale.club_nbr==6212)]

In [21]:
df_no_sale[(df_no_sale.nosales_club_thresh < 0.15)|(df_no_sale.nosales_club_thresh > 0.85)]

,club_nbr,cancelled_club_thresh,nosales_club_thresh
197,6307,0.4411,0.1377
342,6512,0.4146,0.8684
591,8292,0.4372,0.9666
598,8299,0.3785,0.1485


In [20]:
df_no_sale[(df_no_sale.nosales_club_thresh < 0.2)|(df_no_sale.nosales_club_thresh > 0.8)].count()

club_nbr                 20
cancelled_club_thresh    20
nosales_club_thresh      20
dtype: int64

In [ ]:
!pwd

In [5]:
df_databricks_no_sale = pd.read_csv(r'/home/jupyter/oyi-ds-vertex/src/pipeline/databricks_oyi_nosale.csv')
df_databricks_no_sale_22oct14=df_databricks_no_sale[df_databricks_no_sale.update_ts.str.contains('2022-10-14')].reset_index()

In [ ]:
df_databricks_no_sale.info()

In [ ]:
df_no_sale

In [ ]:
df_databricks_no_sale_22oct14

In [6]:
joined_no_sale = df_no_sale.merge(df_databricks_no_sale_22oct14, left_on=["club_nbr"], right_on=["club_nbr"], suffixes=('_left', '_right'))
# joined_no_sale["delta_no_sale"] = joined_no_sale.apply(lambda x: x.nosales_club_thresh - x.nosales)

In [11]:
joined_no_sale

,club_nbr,cancelled_club_thresh,nosales_club_thresh,index,nosales,cancelled,update_ts,nosales_precision,nosales_recall,cancelled_precision,cancelled_recall
0,4041,0.5448,0.5220,27926,0.2786,0.4691,2022-10-14T19:33:44.947Z,0.3596,1.0000,0.6667,0.8571
1,4109,0.5617,0.5857,27927,0.3723,0.4775,2022-10-14T19:33:44.947Z,0.4942,0.9922,0.5455,0.8571
2,4702,0.5333,0.4565,27928,0.4179,0.4634,2022-10-14T19:33:44.947Z,0.5827,0.8804,0.2903,1.0000
3,4703,0.3857,0.3308,27929,0.3577,0.3895,2022-10-14T19:33:44.947Z,0.5362,0.9475,0.7403,0.9913
4,4704,0.3947,0.3207,27930,0.3522,0.3940,2022-10-14T19:33:44.947Z,0.5773,0.9413,0.7286,0.9949
...,...,...,...,...,...,...,...,...,...,...,...
594,8295,0.3840,0.4089,27846,0.3198,0.3891,2022-10-14T19:33:44.947Z,0.5286,0.8952,0.6751,0.9852
595,8296,0.5001,0.5485,27847,0.3994,0.4467,2022-10-14T19:33:44.947Z,0.7240,0.9205,0.8148,0.9565
596,8297,0.2706,0.4746,27848,0.3660,0.3320,2022-10-14T19:33:44.947Z,0.5479,0.7921,0.5263,1.0000
597,8298,0.4145,0.4570,27849,0.4068,0.4039,2022-10-14T19:33:44.947Z,0.6187,0.9247,0.6429,1.0000


In [12]:
joined_no_sale = joined_no_sale.assign(delta_nosales=lambda x: abs(x.nosales_club_thresh - x.nosales))

In [17]:
joined_no_sale[(joined_no_sale.delta_nosales > 0.3)].count()

club_nbr                 19
cancelled_club_thresh    19
nosales_club_thresh      19
index                    19
nosales                  19
cancelled                19
update_ts                19
nosales_precision        19
nosales_recall           19
cancelled_precision      19
cancelled_recall         19
delta_nosales            19
dtype: int64

In [16]:
joined_no_sale[(joined_no_sale.delta_nosales > 0.3)]

,club_nbr,cancelled_club_thresh,nosales_club_thresh,index,nosales,cancelled,update_ts,nosales_precision,nosales_recall,cancelled_precision,cancelled_recall,delta_nosales
34,4763,0.2974,0.5355,27960,0.2057,0.3454,2022-10-14T19:33:44.947Z,0.5271,1.0000,0.6929,1.0000,0.3298
48,4781,0.4651,0.6222,27675,0.2670,0.4292,2022-10-14T19:33:44.947Z,0.4296,0.9945,0.8200,0.9762,0.3552
66,4805,0.3817,0.6166,27693,0.3156,0.3878,2022-10-14T19:33:44.947Z,0.4976,1.0000,0.6106,0.9718,0.3010
75,4824,0.3959,0.6176,28151,0.3039,0.3946,2022-10-14T19:33:44.947Z,0.4792,0.9751,0.4686,0.9798,0.3137
129,4952,0.4324,0.6357,27756,0.3103,0.4119,2022-10-14T19:33:44.947Z,0.4145,0.9877,0.8730,1.0000,0.3254
188,6262,0.4192,0.6674,27590,0.2540,0.4063,2022-10-14T19:33:44.947Z,0.4328,1.0000,0.7113,0.9718,0.4134
234,6350,0.4096,0.6458,28122,0.2116,0.4015,2022-10-14T19:33:44.947Z,0.5259,1.0000,0.8641,0.9889,0.4342
311,6448,0.3807,0.6114,27901,0.2669,0.3901,2022-10-14T19:33:44.947Z,0.4961,1.0000,0.4881,0.8200,0.3445
328,6474,0.3498,0.6391,27918,0.3261,0.3742,2022-10-14T19:33:44.947Z,0.4178,0.9531,0.5319,0.9091,0.3130
342,6512,0.4146,0.8684,27632,0.3952,0.4038,2022-10-14T19:33:44.947Z,0.3892,0.8876,0.3260,1.0000,0.4732


In [22]:
joined_no_sale[(joined_no_sale.delta_nosales > 0.3)].count()

club_nbr                 19
cancelled_club_thresh    19
nosales_club_thresh      19
index                    19
nosales                  19
cancelled                19
update_ts                19
nosales_precision        19
nosales_recall           19
cancelled_precision      19
cancelled_recall         19
delta_nosales            19
dtype: int64